In [ ]:
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras.layers import Conv3D,MaxPooling3D,ZeroPadding3D,Flatten,Dense,Dropout,Convolution3D
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,BatchNormalization,ReLU,MaxPooling2D,Add
from tensorflow.keras.layers import GlobalAveragePooling2D,concatenate,AveragePooling2D
from tensorflow.keras.optimizers import SGD,RMSprop
from tensorflow.keras.losses import categorical_crossentropy,MeanSquaredError,MeanAbsolutePercentageError
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger, TensorBoard
from tensorflow.keras.metrics import TruePositives,FalsePositives,TrueNegatives,FalseNegatives
from tensorflow.keras.metrics import AUC,Accuracy,Precision,Recall

from sklearn.utils import class_weight
import datetime
from tensorflow.keras import Input
from tensorflow.keras import initializers

# Models from scratch

# VGG16

In [ ]:
def vgg16_model(summary=False):
    model = Sequential()
    model.add(Conv2D(input_shape=(256,256,1),filters=64,kernel_size=(3,3), activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3), activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dropout(0.7,seed = seed_value))
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dropout(0.7,seed = seed_value))
    model.add(Dense(units=2, activation="softmax"))
    
    if summary:
        print(model.summary())
    
    return model

# ResNet Variants

In [ ]:
def residual_block(X_start, filters, name, reduce=False, res_conv2d=False):
    """
    Residual building block used by ResNet-50
    """
    nb_filters_1, nb_filters_2, nb_filters_3 = filters
    strides_1 = [2,2] if reduce else [1,1]
        
    X = Conv2D(filters=nb_filters_1, kernel_size=[1,1], strides=strides_1, padding='same', name=name)(X_start)
    X = BatchNormalization()(X)      # default axis-1 is ok
    X = ReLU()(X)
    
    X = Conv2D(filters=nb_filters_2, kernel_size=[3,3], strides=[1,1], padding='same')(X)
    X = BatchNormalization()(X)
    X = ReLU()(X)
    
    X = Conv2D(filters=nb_filters_3, kernel_size=[1,1], strides=[1,1], padding='same')(X)
    X = BatchNormalization()(X)
    
    if res_conv2d:
        X_res = Conv2D(filters=nb_filters_3, kernel_size=[1,1], strides=strides_1, padding='same')(X_start)
        X_res = BatchNormalization()(X_res)
    else:
        X_res = X_start
        
    X = Add()([X, X_res])
    obj = ReLU()
    X = obj(X)
    return X

# ResNet50

In [ ]:
def resnet50(input_shape):
    assert len(input_shape) == 3
    
    X_input = Input(shape=input_shape)

    # conv1
    X = Conv2D(filters=64, kernel_size=[7,7], strides=[2,2], padding='same', name='conv1')(X_input)
    X = BatchNormalization(name='bn_conv1')(X)
    obj = ReLU()
    X = obj(X)
    X = MaxPooling2D([3,3], strides=[2,2])(X)

    # conv2_x
    X = residual_block(X, filters=[64, 64, 256], name='conv2_a', reduce=False, res_conv2d=True)
    X = residual_block(X, filters=[64, 64, 256], name='conv2_b')
    X = residual_block(X, filters=[64, 64, 256], name='conv2_c')

    # conv3_x
    X = residual_block(X, filters=[128, 128, 512], name='conv3_a', reduce=True, res_conv2d=True)
    X = residual_block(X, filters=[128, 128, 512], name='conv3_b')
    X = residual_block(X, filters=[128, 128, 512], name='conv3_c')
    X = residual_block(X, filters=[128, 128, 512], name='conv3_d')

    # conv4_x
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_a', reduce=True, res_conv2d=True)
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_b')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_c')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_d')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_e')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_f')

    # conv5_x
    X = residual_block(X, filters=[512, 512, 2048], name='conv5_a', reduce=True, res_conv2d=True)
    X = residual_block(X, filters=[512, 512, 2048], name='conv5_b')
    X = residual_block(X, filters=[512, 512, 2048], name='conv5_c')

    X = GlobalAveragePooling2D(name='avg_pool')(X)
    X = Flatten()(X)
    X = Dense(units=2, activation='softmax')(X)
    model = tf.keras.models.Model(inputs=X_input, outputs=X)
    return model

# ResNet101


In [ ]:
def resnet101(input_shape):
    assert len(input_shape) == 3
    
    X_input = Input(shape=input_shape)

    # conv1
    X = Conv2D(filters=64, kernel_size=[7,7], strides=[2,2], padding='same', name='conv1')(X_input)
    X = BatchNormalization(name='bn_conv1')(X)
    obj = ReLU()
    X = obj(X)
    X = MaxPooling2D([3,3], strides=[2,2])(X)

    # conv2_x
    X = residual_block(X, filters=[64, 64, 256], name='conv2_a', reduce=False, res_conv2d=True)
    X = residual_block(X, filters=[64, 64, 256], name='conv2_b')
    X = residual_block(X, filters=[64, 64, 256], name='conv2_c')
#     X = GlobalAveragePooling2D(name='max_pool')(X)

    # conv3_x
    X = residual_block(X, filters=[128, 128, 512], name='conv3_a', reduce=True, res_conv2d=True)
    X = residual_block(X, filters=[128, 128, 512], name='conv3_b')
    X = residual_block(X, filters=[128, 128, 512], name='conv3_c')
    X = residual_block(X, filters=[128, 128, 512], name='conv3_d')
#     X = GlobalAveragePooling2D(name='max_pool')(X)

    # conv4_x
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_a', reduce=True, res_conv2d=True)
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_b')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_c')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_d')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_e')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_f')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_g')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_h')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_i')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_j')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_k')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_l')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_m')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_n')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_o')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_p')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_q')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_r')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_s')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_t')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_u')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_v')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_w')
#     X = GlobalAveragePooling2D(name='max_pool')(X)

    # conv5_x
    X = residual_block(X, filters=[512, 512, 2048], name='conv5_a', reduce=True, res_conv2d=True)
    X = residual_block(X, filters=[512, 512, 2048], name='conv5_b')
    X = residual_block(X, filters=[512, 512, 2048], name='conv5_c')

    X = GlobalAveragePooling2D(name='max_pool')(X)
    X = Flatten()(X)
    X = Dense(units=1, activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(units=128, activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(units=1, activation='relu')(X)
    model = tf.keras.models.Model(inputs=X_input, outputs=X)
    return model

# ResNet 152

In [ ]:
def resnet152(input_shape):
    assert len(input_shape) == 3
    
    X_input = Input(shape=input_shape)

    # conv1
    X = Conv2D(filters=64, kernel_size=[7,7], strides=[2,2], padding='same', name='conv1')(X_input)
    X = BatchNormalization(name='bn_conv1')(X)
    obj = ReLU()
    X = obj(X)
    X = MaxPooling2D([3,3], strides=[2,2])(X)

    # conv2_x
    X = residual_block(X, filters=[64, 64, 256], name='conv2_a', reduce=False, res_conv2d=True)
    X = residual_block(X, filters=[64, 64, 256], name='conv2_b')
    X = residual_block(X, filters=[64, 64, 256], name='conv2_c')

    # conv3_x
    X = residual_block(X, filters=[128, 128, 512], name='conv3_a', reduce=True, res_conv2d=True)
    X = residual_block(X, filters=[128, 128, 512], name='conv3_b')
    X = residual_block(X, filters=[128, 128, 512], name='conv3_c')
    X = residual_block(X, filters=[128, 128, 512], name='conv3_d')
    X = residual_block(X, filters=[128, 128, 512], name='conv3_e')
    X = residual_block(X, filters=[128, 128, 512], name='conv3_f')
    X = residual_block(X, filters=[128, 128, 512], name='conv3_g')
    X = residual_block(X, filters=[128, 128, 512], name='conv3_h')

    # conv4_x
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_a', reduce=True, res_conv2d=True)
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_b')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_c')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_d')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_e')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_f')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_g')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_h')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_i')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_j')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_k')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_l')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_m')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_n')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_o')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_p')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_q')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_r')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_s')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_t')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_u')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_v')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_w')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_x')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_y')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_z')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_a1')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_b1')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_c1')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_d1')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_e1')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_f1')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_g1')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_h1')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_i1')
    X = residual_block(X, filters=[256, 256, 1024], name='conv4_j1')

    # conv5_x
    X = residual_block(X, filters=[512, 512, 2048], name='conv5_a', reduce=True, res_conv2d=True)
    X = residual_block(X, filters=[512, 512, 2048], name='conv5_b')
    X = residual_block(X, filters=[512, 512, 2048], name='conv5_c')

    X = GlobalAveragePooling2D(name='avg_pool')(X)
    X = Flatten()(X)
    X = Dense(units=2, activation='softmax')(X)
    model = tf.keras.models.Model(inputs=X_input, outputs=X)
    return model

# UNet

In [ ]:
def get_unet(img_size, num_classes):
     
    inputs = keras.Input(shape=[im_width, im_height,1 ])
  

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='valid')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='vlaid')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(num_classes, 3, activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-3), loss=dice_coef_loss, metrics=[dice_coef])

    return model

# Pre-trained Models

# DenseNet121

In [ ]:
tf.keras.applications.DenseNet121(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
)

# Xception

In [ ]:
tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

# ResNet50V2

In [ ]:
tf.keras.applications.ResNet50V2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

#Inception V3

In [ ]:
tf.keras.applications.InceptionV3(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

# EfficientNet B0

In [ ]:
tf.keras.applications.EfficientNetB0(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    **kwargs
)